In [1]:
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from estimagic_stargazer.stargazer import Stargazer 
from IPython.core.display import HTML, Latex
from tabulate import tabulate
from collections import namedtuple

diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = ['Age', 'Sex', 'BMI', 'ABP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
df['target'] = diabetes.target

est = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:6]])).fit()



C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [2]:
def extract_params_from_sm(model):
    to_concat = []
    params_list = ["params", "pvalues", "bse"]
    for col in params_list:
        to_concat.append(getattr(model, col))
    to_concat.append(model.conf_int()[0])
    to_concat.append(model.conf_int()[1])
    params_df = pd.concat(to_concat, axis=1)
    params_df.columns = ["value", "pvalue", "standard_error", "ci_lower", "ci_upper"]
    return params_df


def extract_info_from_sm(model):
    info = {}
    key_values = [
        "rsquared",
        "rsquared_adj",
        "scale",
        "fvalue",
        "f_pvalue",
        "df_model",
        "df_resid",
    ]
    for kv in key_values:
        info[kv] = getattr(model, kv)
    return info


In [3]:
# use the functions defined above to create estimation result that is namedtuple
namedtuplee = namedtuple("namedtuple", "params info")
est3 = namedtuplee(
params=extract_params_from_sm(est2),
info={**extract_info_from_sm(est2)},
)


In [4]:
# using est3, create dictionary
est4 = {}
est4['params'] = est3.params
est4['info'] = est3.info


In [5]:
type(est4)

dict

In [6]:
# check stargazer works for sns regression result, dictionary and namedtuple
stargazer=Stargazer([est,est4,est3])

TypeError: namedtuple() got an unexpected keyword argument 'params'

In [ ]:
HTML(stargazer.render_html())


In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.title('Diabetes Study')

In [ ]:
HTML(stargazer.render_html())


In [ ]:
type(Latex(stargazer.render_latex()))

In [ ]:
Latex(stargazer.render_latex())


In [ ]:
stargazer.custom_columns(['Model 1','Model 2','Model3'],[1,1,1])

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.custom_columns('Test model name')

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:

stargazer.show_model_numbers(False)

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.significant_digits(2)

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.show_confidence_intervals(True)

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.covariate_order(['BMI', 'Age', 'S1', 'Sex'])

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.rename_covariates({'Age': 'Oldness'})

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.show_degrees_of_freedom(False)

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.add_custom_notes(['First note', 'Second note'])

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.significance_levels([0.1, 0.05, 0.07])

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())

In [ ]:
stargazer.append_notes(False)

In [ ]:
HTML(stargazer.render_html())

In [ ]:
Latex(stargazer.render_latex())